In [1]:
import numpy as np

# Intro to multidisciplinary analysis (MDA)

The motivation for multidisciplinary design optimization (MDO) comes from the large number of multidisciplinary analyses (MDAs) that are common in engineering. The most famous MDA in engineering is the aero-structural analysis problem. We will illustrate using the example of a wing subject to aero-structural loads. The aerodynamic forces deflect the wing which changes its shape, which in turn changes the aerodynamic behaviour of the wing.

We will look at a simplified example of a wing modeled as a cantilever beam, made of Euler-Bernoulli beam sections, subject to aerodynamic forces.

## Illustrative example: Structure-fluids interaction problem

<center>

| <p style="text-align:center;"><img src="./images/wing_structure_fluid.png" alt="wing" title="A cantilever wing" width="550p" align="center"/></p> |
|:--:|
| Fig.0 Aerostructural wing model showing the aerodynamic state variables (circulations $\Gamma$) on the left and structural state variables (displacements $\mathbf{d}_z$ and rotations $\mathbf{d}_\theta$) on the right.|
|*From Martins and Ning*|
</center>

### Structural Analysis

Structural analysis can be performed using finite element software to solve for the structural displacements $\mathbf{d}$ given some boundary conditions such as forces $\mathbf{q}$. These forces may come from an aerodynamic analysis and are related to the computational fluid dynamics state variables $\hat{\Gamma}$. In this example, $\hat{\Gamma}$ is the circulation vortex strength on a wing. The hat denotes that these values are estimates of the state variables when the analysis has converged and are called *coupling variables*.

\begin{equation}
K\mathbf{d} - \mathbf{q}(\hat{\Gamma}) = \mathbf{0}
\tag{1}
\end{equation}

Solving the above equation is done by specialized finite element software and returns a solution ${\mathbf{d}}$ that estimates the displacements. In other words, the FEA program can be thought of as a function $\mathcal{U}_1(\hat{\Gamma})$ that returns the displacements ${\mathbf{d}}$ given some aerodynamic forces $\mathbf{q}(\hat{\Gamma})$. 

The residual for this discipline becomes $r_1(\hat{\mathbf{d}};\hat{\mathbf{q}}) = \hat{\mathbf{d}} - \mathcal{U}_1(\hat{\Gamma}) = \hat{\mathbf{d}} - {\mathbf{d}} = \mathbf{0}$ and solving it returns an estimate of displacements that will cause the analysis to converge $\hat{\mathbf{d}}$. These estimates will be passed back to the aerodynamic analysis.

### Aerodynamic Analysis

The aerodynamic analysis involves solving the following system of equations using computational fluid dynamics (CFD).

\begin{equation}
A(\hat{\mathbf{d}})\Gamma - \mathbf{v}(\hat{\mathbf{d}}) = \mathbf{0}
\tag{2}
\end{equation}

where $\Gamma$ is the state vector we want to solve for representing the circulation vortex strength. $\mathbf{v}$ is a vector of boundary conditions (similar to the vector of applied loads $\mathbf{q}$ in the structural analysis). $A$ is the matrix of aerodynamic influence coefficients (similar to the global stiffness matrix $K$ from the structural analysis). However, unlike $K$, $A$ depends on the shape of the wing which in turn depends on the displacements estimated by the structural analysis $\hat{\mathbf{d}}$.

The residual for this discipline becomes $r_2(\hat{\Gamma};\hat{\mathbf{d}}) = \hat{\Gamma} - \mathcal{U}_2(\hat{\mathbf{d}}) = \hat{\Gamma} - {\Gamma} = \mathbf{0}$, where $\mathcal{U}_2(\hat{\mathbf{d}})$ involves solving Equation (2) for the circulation vortex strength $\Gamma$.

The estimated coupling variable $\hat{\Gamma}$ is passed back to the aerodynamic analysis. The previous analysis can be described by the diagram below.

<center>

| <p style="text-align:center;"><img src="./images/MDA_diagram_aero.png" alt="mda_diag" width="750p" align="center"/></p> |
|:--:|
| Fig.1 Aerostructural multidisciplinary analysis (MDA) diagram.|
</center>



The XDSM for the aforementioned aero-structural analysis problem is shown below:

The XDSM for aerostructural example looks like this:

<center>

| <p style="text-align:center;"><img src="./xdsm/aerostructural.png" alt="aero" width="750p" align="center"/></p> |
|:--:|
| Fig.2 XDSM for Aerostructural multidisciplinary analysis (MDA).|
</center>

## Numerical MDA example

Consider the following coupled optimization problem

\begin{equation*}
	\begin{aligned}
		& \underset{\mathbf{x} = \left[u,v,w\right]^\mathrm{T}}{\text{minimize}}
		& & f(u,v;a,b) = u+v+a+b\\
		& \text{subject to}
		& & g(w;b) = w + b -10 \leq 0\\
        & & & 0 \leq u,v,w \leq 10\\
		& \text{while solving}
		& & r_1(a;b) = a - \left(\log(u) + \log(v) + \log(b)\right) = 0\\
        & & & r_2(b;a) = b - \left(u^{-1} + w^{-1} + a^{-1}\right) = 0\\
		& \text{for a given $u$, $v$, and $w$}
	\end{aligned}
    \tag{3}
\end{equation*}

Solving for the state variables $a$ and $b$ involves solving the set of equations below. Equation (4) is an example of an MDA (just like the aerostructural problem earlier):

\begin{equation*}
	\begin{aligned}
		& r_1(a;b) = a - \left(\log(u) + \log(v) + \log(b)\right) = 0\\
		& r_2(b;a) = b - \left(u^{-1} + w^{-1} + a^{-1}\right) = 0\\
	\end{aligned}
    \tag{4}
\end{equation*}

which cannot be solved explicitly. To prove this, Let us try to find an explicit expression for $a$ in terms of $b$.

In [2]:
import sympy as sym
from sympy import pprint, latex, root
from IPython.display import display, Latex

a = sym.Symbol("a")
b = sym.Symbol("b")
u = sym.Symbol("u")
v = sym.Symbol("v")
w = sym.Symbol("w")

r1 = a - sym.log(u) - sym.log(v) - sym.log(b)
r2 = b - 1/u - 1/w  - 1/a

eq1 = a - sym.log(u) - sym.log(v) - sym.log(1/u + 1/w  + 1/a)
# sym.solve(eq1,a) # trying to solve for a explicitly will result in an error!

Instead, we focus on solving for $a$ and $b$ such that the residuals $r_1=0$ and $r_2=0$ iteratively using a Multidisciplinary Analysis (MDA) for a fixed value of $u$, $v$, and $w$. First let us slightly change the notation of the governing equations:

\begin{align*}
	& r_1(\hat{a};\hat{b}) = \hat{a} - \mathcal{U}_1(\hat{b}) = 0\\
	& r_2(\hat{b};\hat{a}) = \hat{b} - \mathcal{U}_2(\hat{a}) = 0\\
\end{align*}

$\mathcal{U}_1(\hat{b})$ and $\mathcal{U}_2(\hat{a})$ are considered the discipline solvers and could represent the FEA and CFD programs in the aerostructural problem.

## The Nonlinear block Gauss–Seidel and block Jacobi MDA algorithms

The MDA approach we will use is called the *block Gauss–Seidel algorithm* and has an XDSM similar to that of the aerostructural problem. 
1) Starting with an initial guess for $\hat{a}$ and $\hat{b}$, $\hat{a}^{(0)}$ and $\hat{b}^{(0)}$, respectively, 
2) we solve discipline 1 for $\hat{a}$.
3) We feed $\hat{a}$ into discipline 2,
4) and then solve for $\hat{b}$ and then go back to step 2.

We repeat the above process until $\hat{a}_\mathrm{new} - \hat{a}_\mathrm{old} \leq \varepsilon$, where $\varepsilon$ is some tolerance. We can also solve for $\hat{a}$ and $\hat{b}$ simultaneously rather than serially resulting in the block Jacobi MDA.

The XDSM for the above MDAs is shown below:

<center>

| <p style="text-align:center;"><img src="./xdsm/gauss.png" alt="gauss" width="750p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./xdsm/jacobi.png" alt="jacobi" width="750p" align="center"/></p> |
| Fig.3 block Gauss–Seidel and Block Jacobi MDAs.|
</center>

This algorithm is implemented below in Python.

In [3]:
# these are your coupled disciplines
U1 = lambda uj: np.log(u) + np.log(v) + np.log(uj[0]+1e-6)
U2 = lambda uj: 1/u + 1/w + 1/(uj[0]+1e-6)

u = 1
v = 1
w = 1
a0=1
b0=1
uk = np.array([a0,b0])
k = 0
e = 1e-6
disciplines = [U1,U2]

while True:
    uk_old = uk.copy()
    for i,discipline in enumerate(disciplines):
        mask = [True,]*len(disciplines)
        mask[i] = False # select all state variables except the current one (u_i)
        uk[i] = discipline(uk[mask])
    if np.linalg.norm(uk - uk_old) <= e or k > 10:
        break
    k+=1
    print(uk)

[      0 1000002]
[13  2]
[      0 1000002]
[13  2]
[      0 1000002]
[13  2]
[      0 1000002]
[13  2]
[      0 1000002]
[13  2]
[      0 1000002]


We can see that the previous algorithm did not converge. It started to oscillate.
We can use an alternative algorithm that is based on the Newton-Raphson scheme to handle the coupling between the disciplines.

## The reduced-space hierarchical Newton solver

This algorithm is unequivocally also called *Newton’s Method*. As with the Newton-Raphson algorithm, it requires derivative information to find the coupling variables $\hat{a}$ and $\hat{b}$ at which convergence occurs. The algorithm goes something like this:

1) Starting with an initial guess for $\hat{a}$ and $\hat{b}$, $\hat{a}^{(0)}$ and $\hat{b}^{(0)}$, respectively, 
2) we solve discipline 1 for $\hat{a}$. We also find the gradient of the discipline analysis $\mathcal{U}(\hat{b})$ with respect to $\hat{b}$. In other words, $\dfrac{\partial\mathcal{U}_1}{\partial\hat{b}}$ (all coupling variables except $\hat{a}$)
3) In parallel, we solve discipline 2 for $\hat{b}$. We also find the gradient with respect to $\hat{a}$, $\dfrac{\partial\mathcal{U}_2}{\partial\hat{b}}$ (all coupling variables except $\hat{b}$)
4) We solve the following system of equations for $\Delta\hat{a}$ and $\Delta\hat{b}$:
$$
\begin{bmatrix}
    1       & \left.\dfrac{\partial\mathcal{U}_1}{\partial\hat{b}}\right|_{\hat{b}}  \\
    \left.\dfrac{\partial\mathcal{U}_2}{\partial\hat{a}}\right|_{\hat{a}}        & 1  \\
\end{bmatrix}
\begin{bmatrix}
    \Delta\hat{a}\\
    \Delta\hat{b}\\
\end{bmatrix} = -
\begin{bmatrix}
    \hat{a} - \mathcal{U}_1(\hat{b})\\
    \hat{b} - \mathcal{U}_2(\hat{a})\\
\end{bmatrix}
$$
5) we update $\hat{a}$ and $\hat{b}$ according to $\hat{a} \gets \hat{a} + \Delta\hat{a}$ and $\hat{a} \gets \hat{a} + \Delta\hat{a}$, respectively and go back to step 2.

The XDSM for the above MDA is shown below:

<center>

| <p style="text-align:center;"><img src="./xdsm/newton.png" alt="newton" width="550p" align="center"/></p> |
|:--:|
| Fig.4 Reduced-space hierarchical Newton solver.|
</center>

This algorithm is implemented below in Python.

In [4]:
# these are your coupled disciplines
U1 = lambda uj: np.log(u) + np.log(v) + np.log(uj[0])
U2 = lambda uj: 1/u + 1/w + 1/(uj[0]+1e-6)

# These are the Jacobians of your disciplines
dU1 = lambda uj: np.array([1/(uj[0]+1e-6)])
dU2 = lambda uj: np.array([-1/(uj[0]+1e-6)**2])

u = 1
v = 1
w = 1
a0=100
b0=100
uk = np.array([a0,b0])
k = 0
e = 1e-6
disciplines = [U1,U2]
Jacobians = [dU1,dU2]

while True:
    U = np.zeros_like(uk)
    drdU = np.ones((len(uk),len(uk)))
    for i,(discipline,Jacobian) in enumerate(zip(disciplines,Jacobians)):
        mask = [True,]*len(disciplines)
        mask[i] = False # select all state variables but current one
        U[i] = discipline(uk[mask]) # estimate coupling variables
        drdU[i,mask] = -Jacobian(uk[mask]) # estimate gradient of coupling variables

    r = uk - U # residuals
    if np.linalg.norm(r) <= e or k > 100:
        break
    
    # Newton-Raphson step
    du = np.linalg.inv(drdU) @ -r
    uk = uk + du
    k+=1
    print(uk)

[3.02009699 2.00969799]
[0.96977098 2.55590709]
[1.07893819 2.91509142]
[1.07507004 2.93015913]
[1.07506292 2.93017725]


We found a solution for $a$ and $b$. You can plug those values into your calculator and check that they satisfy the Equations in (4) yourself! 

However, we had to provide the Jacobian of the analysis functions $\dfrac{\partial\mathcal{U}}{\partial\hat{\mathbf{u}}}$ with respect to the coupling variables $\hat{\mathbf{u}} = \begin{bmatrix} \hat{a} & \hat{b} \end{bmatrix}^\mathrm{T}$ for the solver to converge. When $\mathcal{U}_1$ and $\mathcal{U}_2$ are blackboxes (as in the aerostructural problem at the beginning of this notebook), we seldom have access to residuals, let alone gradient information! In other words, we sort of cheated. 

# Multidisciplinary design optimization

Since the MDA seems to converge when using the reduced Newton solver, we will use it to solve the optimization problem (1) at the beginning of this notebook. Solving the optimization problem using an MDA is known as Multidisciplinary feasible (MDF) approach.

## Multidisciplinary feasible (MDF) optimization

\begin{equation*}
	\begin{aligned}
		& \underset{u,v,w}{\text{minimize}}
		& & f(u,v,w) = u+v+\hat{a}(u,v,w)+\hat{b}(u,v,w)\\
		& \text{subject to}
		& & g(u,v,w) = w + \hat{b}(u,v,w) -10 \leq 0\\
        & & & 0 \leq u,v,w \leq 10\\
	\end{aligned}
    \tag{5}
\end{equation*}


The above optimization problem is called a Multi-disciplinary feasible (MDF) MDO problem since it relies on a monolithic MDA. Note that we have switched out $a$ and $b$ by their MDA counterparts $\hat{a}$ and $\hat{b}$ which need to be solved for during every optimization loop!

We can use a blackbox optimization algorithm to solve the problem, where the blackbox in this case is the MDA we developed in the previous section. We define the blackbox below.

In [5]:
# these are your coupled disciplines
U1 = lambda x,uj: np.log(x[0]+1e-6) + np.log(x[1]+1e-6) + np.log(uj[0]+1e-6)
U2 = lambda x,uj: 1/(x[0]+1e-6) + 1/(x[2]+1e-6) + 1/(uj[0]+1e-6)

# These are the Jacobians of your disciplines
dU1 = lambda x,uj: np.array([1/(uj[0]+1e-6)])
dU2 = lambda x,uj: np.array([-1/(uj[0]+1e-6)**2])

def my_MDA(x,u0,disciplines,Jacobians,epsilon=1e-6,k_max=100):
    uk = u0
    k = 0
    disciplines = [U1,U2]
    Jacobians = [dU1,dU2]
    while True:
        U = np.zeros_like(uk)
        drdU = np.ones((len(uk),len(uk)))
        for i,(discipline,Jacobian) in enumerate(zip(disciplines,Jacobians)):
            mask = [True,]*len(disciplines)
            mask[i] = False # select all state variables but current one
            U[i] = discipline(x,uk[mask]) # estimate coupling variables
            drdU[i,mask] = -Jacobian(x,uk[mask]) # estimate gradient of coupling variables

        r = uk - U # residuals
        if np.linalg.norm(r) <= epsilon or k > k_max:
            break
        
        # Newton-Raphson step
        du = np.linalg.inv(drdU) @ -r
        uk = uk + du
        k+=1

    return uk

u = 0.0
v = 0.0
w = 0.0
x = [u,v,w]
state0=np.array([100,100])
disciplines = [U1,U2]
Jacobians = [dU1,dU2]

u_hat = my_MDA(x,state0,disciplines,Jacobians)
print(u_hat)

[-1.31223634e+01  1.99999992e+06]


A blackbox that returns the objective and constraint value can now be defined as follows:

In [6]:
def my_bb(x,args):

    state0 = args[0]
    disciplines = args[1]
    Jacobians = args[2]

    u_hat = my_MDA(x,state0,disciplines,Jacobians)
    f = x[0] + x[1] + u_hat[0] + u_hat[1]
    g = x[2] + u_hat[1] - 10.0

    return [f,[g,-u_hat[0],-u_hat[1]]]

The blackbox above is what we can feed to [`OMADS`](https://ahmed-bayoumy.github.io/OMADS/).

In [7]:
import OMADS

# Optimization setup
u0 = 1.0; v0 = 1.0; w0 = 1.0
x0 = [u0,v0,w0]

f,g = my_bb(x0,args=[state0,disciplines,Jacobians]) # its a good idea to test the blackbox function first at the initial guess

lb = [0.0, 0.0, 0.0]
ub = [10., 10., 10.]

eval = {"blackbox": my_bb,
        "constants": [state0,disciplines,Jacobians]} # you can pass blackbox specific options using the ``constants`` key
param = {"baseline": x0,
        "lb": lb,
        "ub": ub,
        "var_names": ["u", "v", "w"],
        "scaling": 10.0,
        "post_dir": "./post"} # these are OMADS specific options. You can modify them as necessary
options = {"seed": 0, "budget": 100000, "tol": 1e-12, "display": False}

data = {"evaluator": eval, "param": param, "options":options}

out = {}
# out is a dictionary that will hold output data of the final solution. The out dictionary has three keys: "xmin", "fmin" and "hmin"

out = OMADS.main(data)
fopt,gopt = my_bb(out["xmin"],args=[state0,disciplines,Jacobians]) # evaluate blackbox at optimizer
uopt = my_MDA(out["xmin"],state0,disciplines,Jacobians) # evaluate MDA at optimizer

import json
for key,value in out.items():
    print(key," : ",value)
print("===========================")
f_tex = r"$f(\mathbf{x}^*) = %.4f$" %(out["fmin"])
display(Latex(f_tex)) # minimum
x_tex = r"$\mathbf{x}^*$ = $[%.4f~~%.4f~~%.4f]^\mathrm{T}$" %(out["xmin"][0],out["xmin"][1],out["xmin"][2])
display(Latex(x_tex)) # optimizer
g_tex = r"$g(\mathbf{x}^*)$ = $%.4f$" %(gopt[0])
display(Latex(g_tex)) # constraints
u_tex = r"$\hat{\mathbf{u}}(\mathbf{x}^*)$ = $[%.4f~~%.4f]^\mathrm{T}$" %(uopt[0],uopt[1])
display(Latex(u_tex)) # constraints
# print(f_tex)
# print(x_tex)
# print(g_tex)
# print(u_tex)

xmin  :  [1.1874999999927240424, 0.74511669576168060303, 7.6562655009329319]
fmin  :  5.005735228721652552
hmin  :  9.996378214847634363e-19
nbb_evals  :  469
niterations  :  78
nb_success  :  27
psize  :  9.094947017729282379e-13
psuccess  :  7.2759576141834259033e-12
pmax  :  2.0


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We can see that we obtained a solution to the optimization problem in Equation (5). This entire process above is known as MDF. You should resort to MDF if you manage to find an MDA that converges robustly. i.e., for every possible value of the optimization variables $\mathbf{x}\in\mathcal{F}$, where $\mathcal{F}$ is your feasible design space that satisfies all your constraints. 

The XDSM diagram and its corresponding block diagram for the MDF optimization we just performed is given below:

<center>

| <p style="text-align:center;"><img src="./xdsm/mdf.png" alt="mdf" width="950p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./images/MDF_diagram_numerical.png" alt="mdf" width="950p" align="center"/></p> |
| Fig.5 MDF using a reduced-space hierarchical Newton MDA.|
</center>

or more compactly and generally, as follows:

<center>

| <p style="text-align:center;"><img src="./xdsm/mdf_newton_compact.png" alt="mdf_c" width="850p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./images/MDF_diagram_newton.png" alt="mdf_diag" width="850p" align="center"/></p> |
| Fig.6 MDF using a reduced-space hierarchical Newton MDA (generic problem).|
</center>

However, as mention earlier we cheated by using the coupled gradients $\dfrac{\partial\mathcal{U}}{\partial\hat{\mathbf{u}}}$. We will now try to solve the above problem without having to resort to an MDA altogether.

## Individual disciplinary feasible (IDF) optimization

We can do away with MDA by letting the optimization algorithm handle to the convergence of the disciplines (i.e., coordinate their solutions). This is done by creating new variables $\mathbf{u}^t$ known as *target variables*. They are essentially a copy of the actual coupling variables. We then introduce consistency constraints $\mathbf{h}_c = \mathbf{u}^t - \mathbf{u} = \mathbf{0}$ to the original optimization problem. The original optimization problem in Equation (3) becomes:

\begin{equation*}
	\begin{aligned}
		& \underset{u,v,w,\hat{a}^t,\hat{b}^t}{\text{minimize}}
		& & f(\mathbf{x};\hat{a},\hat{b}) = u+v+\hat{a}+\hat{b}\\
		& \text{subject to}
		& & g(\mathbf{x};\hat{b}) = w + \hat{b} -10 \leq 0\\
		& & & h_1^c(\hat{a}^t;\hat{a}) = \hat{a}^t - \hat{a} = 0\\
		& & & h_2^c(\hat{b}^t;\hat{b}) = \hat{b}^t - \hat{b} = 0\\
        & & & 0 \leq u,v,w \leq 10\\
		& \text{while solving}
		& & r_1(\hat{a};\mathbf{x},\hat{b}^t) = \hat{a} - \left(\log(u) + \log(v) + \log(\hat{b}^t)\right) = 0\\
        & & & r_2(\hat{b};\mathbf{x},\hat{a}^t) = \hat{b} - \left(\dfrac{1}{u} + \dfrac{1}{w} + \dfrac{1}{\hat{a}^t}\right) = 0\\
		& \text{for a given $u$, $v$, $w$, $\hat{a}^t$ and $\hat{b}^t$}
	\end{aligned}
    \tag{6}
\end{equation*}

Notice that the optimization algorithm must now control 5 variables simultaneously and has two additional equality constraints to worry about. If those copies of the coupling variables were displacement and vortex fields from the aerostructural problem, the optimization algorithm must control tens of thousands of variables and might go bananas.

For now since our problem is small, let us attempt to solve it again using `OMADS`. We first define the blackbox associated with IDF.

In [8]:
def my_bb(x,args):

    at = x[3]; bt = x[4] # target variables
    ut = np.array([at,bt])
    disciplines = args[0]

    # solve for coupling variables
    u_hat = np.zeros(len(disciplines))
    for i,discipline in enumerate(disciplines):
        mask = [True,]*len(disciplines)
        mask[i] = False # select all coupling variables but current one
        u_hat[i] = discipline(x,ut[mask]) # estimate coupling variables

    f = x[0] + x[1] + u_hat[0] + u_hat[1]
    g = x[2] + u_hat[1] - 10.0
    gc = np.linalg.norm(ut - u_hat) - 1e-6

    return [f,[g,gc]]

Note that I had to slightly reformulate the consistency constraints $h_1^c$ and $h_2^c$ as an inequality constraint ${g}^c = \left|\left|\mathbf{u}^t - \mathbf{u}\right|\right| - \varepsilon \leq 0$ since `OMADS` does not yet support equality constraints.

In [9]:
# Optimization setup
u0 = 1.0; v0 = 1.0; w0 = 1.0; a0 = 1.0; b0 = 1.0
x0 = [u0,v0,w0,a0,b0]

f,g = my_bb(x0,args=[disciplines]) # its a good idea to test the blackbox function first at the initial guess

lb = [0.0, 0.0, 0.0, 0.0, 0.0]
ub = [10., 10., 10., 10., 10.]

eval = {"blackbox": my_bb,
        "constants": [disciplines]} # you can pass blackbox specific options using the ``constants`` key
param = {"baseline": x0,
        "lb": lb,
        "ub": ub,
        "var_names": ["u", "v", "w", "a^t", "b^t"],
        "scaling": 10.0,
        "post_dir": "./post"} # these are OMADS specific options. You can modify them as necessary
options = {"seed": 0, "budget": 100000, "tol": 1e-12, "display": False}

data = {"evaluator": eval, "param": param, "options":options}

out = {}
# out is a dictionary that will hold output data of the final solution. The out dictionary has three keys: "xmin", "fmin" and "hmin"

out = OMADS.main(data)
fopt,gopt = my_bb(out["xmin"],args=[disciplines]) # evaluate blackbox at optimizer
uopt = my_MDA(out["xmin"][:3],state0,disciplines,Jacobians) # evaluate MDA at optimizer to check the true coupling variables

import json
for key,value in out.items():
    print(key," : ",value)
print("===========================")
f_tex = r"$f(\mathbf{x}^*) = %.4f$" %(out["fmin"])
display(Latex(f_tex)) # minimum
x_tex = r"$\mathbf{x}^*$ = $[%.4f~~%.4f~~%.4f]^\mathrm{T}$" %(out["xmin"][0],out["xmin"][1],out["xmin"][2])
display(Latex(x_tex)) # optimizer
g_tex = r"$g(\mathbf{x}^*)$ = $%.4f$" %(gopt[0])
display(Latex(g_tex)) # constraints
gc_tex = r"$g_c(\mathbf{x}^*)$ = $%.4f$" %(gopt[1])
display(Latex(gc_tex)) # inconsistency
ut_tex = r"$\hat{\mathbf{u}}^t$ = $[%.4f~~%.4f]^\mathrm{T}$" %(out["xmin"][3],out["xmin"][4])
display(Latex(ut_tex)) # target states
u_tex = r"$\hat{\mathbf{u}}$ = $[%.4f~~%.4f]^\mathrm{T}$" %(uopt[0],uopt[1])
display(Latex(u_tex)) # states
# print(f_tex)
# print(x_tex)
# print(g_tex)
# print(gc_tex)
# print(ut_tex)
# print(u_tex)

xmin  :  [1.0, 1.20812225341796875, 3.999998461147697526, 1.0, 2.249998331069946289]
fmin  :  5.458119331294737586
hmin  :  9.9995692311506984225e-19
nbb_evals  :  1261
niterations  :  126
nb_success  :  46
psize  :  9.094947017729282379e-13
psuccess  :  1.8189894035458564758e-12
pmax  :  4.0


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We converged to a solution thats worse than the one obtained through MDF because of the larger number of variables and constraints that the optimizer has to handle. Imagine what would happen if we tried solving the aerostructural program using IDF.

However, there is a silver lining. The solution is a feasible one since the target variables $\hat{\mathbf{u}}^t$ match the true coupling variables $\hat{\mathbf{u}}$ obtained from the MDA.

The XDSM for the IDF optimization approach and its corresponding block diagram is shown below.

<center>

| <p style="text-align:center;"><img src="./xdsm/idf.png" alt="idf" width="950p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./images/IDF_diagram_numerical.png" alt="idf_diag" width="950p" align="center"/></p> |
| Fig.7 IDF architecture of the numerical example.|
</center>

The general IDF architecture is given by the following XDSM and block diagrams:

<center>

| <p style="text-align:center;"><img src="./xdsm/idf_compact.png" alt="idf_c" width="850p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./images/IDF_diagram.png" alt="idf_diag" width="850p" align="center"/></p> |
| Fig.8 IDF architecture for a generic problem.|
</center>

## Distributed MDO

The state-of-the-art in multidisciplinary design optimization (MDO) is known as distributed MDO (DMDO). There are a plethora if DMDO architectures out there. We will focus on a simple one first introduced by [Kim et al. (2003)](https://asmedigitalcollection.asme.org/mechanicaldesign/article-abstract/125/3/481/476066/Analytical-Target-Cascading-in-Automotive-Vehicle?redirectedFrom=fulltext) and later formalized as an MDO architecture by [Tosserams et al. (2006)](https://link.springer.com/article/10.1007/s00158-005-0579-0). 

The MDO architecture referenced above is known as *analytical target cascading (ATC)*. In this architecture we use a divide and conquer strategy by splitting up our optimization problem into a system level problem and several discipline subproblems. Notice that in IDF, and MDF we still solved only one optimization problem.

We first must introduce the notion of *shared variables* $\mathbf{x}_0$ and *local variables* $\mathbf{x}_i$ which are local to discipline $i$. This means that our vector of design variables becomes $\mathbf{x} = \begin{bmatrix} \mathbf{x}_0^\mathrm{T} & \mathbf{x}_1^\mathrm{T} & \cdots & \mathbf{x}_m^\mathrm{T} \end{bmatrix}$. 

We can do the same thing for constraints. We *nonlocal constraints* $\mathbf{g}_0$ and *local constraints* $\mathbf{g}_i$ for discipline $i$. This means that our vector of design constraints becomes $\mathbf{g} = \begin{bmatrix} \mathbf{g}_0^\mathrm{T} & \mathbf{g}_1^\mathrm{T} & \cdots & \mathbf{g}_m^\mathrm{T} \end{bmatrix}$. 

**System-level optimization problem**

In ATC, the system level problem modifies the original problem as
follows: 
1) local constraints are removed, 
2) target coupling variables $\mathbf{u}^t$ are added as design variables (just like IDF), and 
3) a penalty term (similar to the consistency constraints $g_c$ we saw earlier in IDF) to the system-level objective function $f_0$ for violating the consistency constraints.

\begin{equation*}
	\begin{aligned}
		& \underset{\mathbf{x}_0,\hat{\mathbf{u}}^t}{\text{minimize}}
		& & f_0(\mathbf{x}_0;\mathbf{x}_1,\cdots,\mathbf{x}_m,\hat{\mathbf{u}}^t) + \sum_{i=1}^m \Phi_i(\mathbf{x}_{0,i}^t - \mathbf{x}_{0}, \hat{\mathbf{u}}_{i}^t - \hat{\mathbf{u}}_{i}(\mathbf{x}_0,\mathbf{x}_i,\hat{\mathbf{u}}^t))\\
		& \text{subject to}
		& & \mathbf{g}_{0}(\mathbf{x}_0;\mathbf{x}_1,\cdots,\mathbf{x}_m,\hat{\mathbf{u}}^t) \leq \mathbf{0}
	\end{aligned}
    \tag{7}
\end{equation*}

going back to our numerical example, we can see that $u$ is a shared variable, while $v$ and $w$ are discipline variables. The system level problem gets a copy of the coupling variable estimates from disciplines 2 and 1 as $\hat{a}^t$ and $\hat{b}^t$, respectively, while it gets the actual estimates $\hat{a}$ and $\hat{b}$ from disciplines 1 and 2, respectively.

\begin{align*}
\mathbf{x}_0 & = \left[u\right]^\mathrm{T}\\
\mathbf{x}_1 & = \left[v\right]^\mathrm{T}\\
\mathbf{x}_2 & = \left[w\right]^\mathrm{T}\\
\hat{\mathbf{u}}_1 & = \left[\hat{a}\right]^\mathrm{T}\\
\hat{\mathbf{u}}_2 & = \left[\hat{b}\right]^\mathrm{T}\\
\hat{\mathbf{u}}_1^t & = \left[\hat{a}^t\right]^\mathrm{T}\\
\hat{\mathbf{u}}_2^t & = \left[\hat{b}^t\right]^\mathrm{T}\\
\end{align*}

There are no shared constraints $\mathbf{g}_0$. $f_0$ is our objective function. $f_0(u;v,\hat{a}^t,\hat{b}^t) = u+v+\hat{a}^t+\hat{b}^t$. The system level problem for our example in this notebook becomes:

\begin{equation*}
	\begin{aligned}
		& \underset{u,\hat{a}^t,\hat{b}^t}{\text{minimize}}
		& & u+v+\hat{a}^t+\hat{b}^t + \Phi_1(u_1^t - u, \hat{a}^t - \hat{a}(u,v,\hat{b}^t)) + \Phi_2(u_2^t - u, \hat{b}^t - \hat{b}(u,w,\hat{a}^t))
	\end{aligned}
    \tag{8}
\end{equation*}

Note that everything with a subscript of $0$ belongs to the system-level problem, while $i\neq 0$ belongs to the individual sub-system problems which we will talk about now.

**Sub-system optimization problem(s)**

In ATC, the sub-system level problems have their own local objectives $f_i$ and constraints $g_i$, local variables $\mathbf{x}_i$, and target variables $\hat{\mathbf{u}}_i^t$. They also have their individual disciplinary analyses $r_i = \hat{\mathbf{u}}_i - \mathcal{U}_i = \mathbf{0}$ which need to be satisfied. The $i$-th sub-system optimization problem is formulated as

\begin{equation*}
	\begin{aligned}
		& \underset{\mathbf{x}_{0,i}^t,\mathbf{x}_i}{\text{minimize}}
		& & f_i(\mathbf{x}_{0,i}^t,\mathbf{x}_i;\hat{\mathbf{u}}_i) + \Phi_i(\hat{\mathbf{u}}_{i}^t - \hat{\mathbf{u}}_{i},\mathbf{x}_{0,i}^t - \mathbf{x}_{0})\\
		& \text{subject to}
		& & \mathbf{g}_i(\mathbf{x}_{0,i}^t,\mathbf{x}_i;\hat{\mathbf{u}}_i) \leq \mathbf{0}\\
		& \text{while solving}
		& & r_i(\hat{\mathbf{u}}_i;\mathbf{x}_{0,i}^t,\mathbf{x}_i,\hat{\mathbf{u}}_{j\neq i}^t) = \mathbf{0}\\
		& \text{for}
		& & \hat{\mathbf{u}}_i\\
	\end{aligned}
    \tag{9}
\end{equation*}

Going back to our numerical example, for discipline 1, we have no local objectives $f_1$ or constraints $\mathbf{g}_1$. It will get a copy of the shared variable $u$ as $u_1^t$. It has its own local design variables $v$, and its own coupling variable estimate $\hat{a}$ obtained by solving $\hat{a} - \mathcal{U}_1(u_1^t,v,\hat{b}^t)=0$. It gets a copy of the coupling variables from the system level problem (0) as $\hat{a}^t$ and $\hat{b}^t$. So now we have

\begin{align*}
\mathbf{x}_{0,1}^t & = \left[u_1^t\right]^\mathrm{T}\\
\mathbf{x}_1 & = \left[v\right]^\mathrm{T}\\
\hat{\mathbf{u}}_1 & = \left[\hat{a}\right]^\mathrm{T}\\
\hat{\mathbf{u}}_1^t & = \left[\hat{a}^t\right]^\mathrm{T}\\
\hat{\mathbf{u}}_2^t & = \left[\hat{b}^t\right]^\mathrm{T}\\
\end{align*}

and sub-system 1's optimization problem becomes

\begin{equation*}
	\begin{aligned}
		& \underset{u_1^t,v}{\text{minimize}}
		& & \Phi_1(\hat{a}^t - \hat{a},u_1^t - u)\\
		& \text{while solving}
		& & r_1(\hat{a};u_1^t,v,\hat{b}^t) = \hat{a} - \mathcal{U}_1(u_1^t,v,\hat{b}^t) = 0\\
		& \text{for}
		& & \hat{a}\\
	\end{aligned}
    \tag{10}
\end{equation*}

For discipline 2, we have no local objectives $f_2$. We have a local constraint $\mathbf{g}_2 = \left[g({u}_2^t,w,\hat{b})\right]^\mathrm{T}$. Discipline 2 will get a copy of the shared variable $u$ as $u_2^t$. It has its own local design variables $w$, and its own coupling variable $\hat{b}$ obtained by solving $\hat{b} - \mathcal{U}_2(u_2^t,w,\hat{a}^t)=0$. It gets a copy of the coupling variables from the system level problem (0) as $\hat{a}^t$ and $\hat{b}^t$. So now we have

\begin{align*}
\mathbf{g}_2 & = \left[g(\hat{u}_2^t,w,\hat{b})\right]^\mathrm{T}\\
\mathbf{x}_{0,2}^t & = \left[u_2^t\right]^\mathrm{T}\\
\mathbf{x}_2 & = \left[w\right]^\mathrm{T}\\
\hat{\mathbf{u}}_2 & = \left[\hat{b}\right]^\mathrm{T}\\
\hat{\mathbf{u}}_1^t & = \left[\hat{a}^t\right]^\mathrm{T}\\
\hat{\mathbf{u}}_2^t & = \left[\hat{b}^t\right]^\mathrm{T}\\
\end{align*}

and sub-system 2's optimization problem becomes

\begin{equation*}
	\begin{aligned}
		& \underset{u_2^t,w}{\text{minimize}}
		& & \Phi_2(\hat{b}^t - \hat{b},u_2^t - u)\\
		& \text{subject to}
		& & g({u}_2^t,w,\hat{b}) = w + \hat{b} - 10 \leq 0\\
		& \text{while solving}
		& & r_2(\hat{b};u_2^t,w,\hat{a}^t) = \hat{b} - \mathcal{U}_2(u_2^t,w,\hat{a}^t) = 0\\
		& \text{for}
		& & \hat{b}\\
	\end{aligned}
    \tag{11}
\end{equation*}


The problem formulation can be represented using the following block diagram:

<center>

| <p style="text-align:center;"><img src="./images/ATC_diagram_numerical.png" alt="atc" width="750p" align="center"/></p> |
|:--:|
| Fig.9 Schematic of the numerical example's ATC problem formulation.|
</center>

We can now begin implementing ATC. We have to define a total of 3 blackboxes. One for the system level problem, and two for each discipline. The ATC algorithm also maintains and updates a vector of weights $\mathbf{w}_i$ and $\mathbf{v}_i$ which are associated with each penalty function $\Phi_i$. These weights are continuously updated during ATC iterations. The ATC algorithm is described by these steps:

0) Initiate main ATC iteration\
**repeat**\
**for** Each discipline **do**

1) Initiate discipline optimizer\
	**repeat**

	2) Evaluate disciplinary analysis
	3) Compute discipline objective and constraint functions and
	penalty function values
	4) Update discipline design variables

	**until** 4 $\rightarrow$ 2: Discipline optimization has converged	
		
**end for**

5) Initiate system optimizer\
	**repeat**

	6) Compute system objective, constraints, and all penalty functions
	7) Update system design variables and coupling targets.
		
	**until** 7 $\rightarrow$ 6: System optimization has converged

8) Update penalty weights
**until** 8 $\rightarrow$ 1: maximum inconsistency $\left|\left|\mathbf{g}^c\right|\right|_\infty \leq \varepsilon$

The implementation of this quite tedious and my own implementation of ATC can be found in the file [atc.py](atc.py).

In [10]:
from atc import ATC

# these are your coupled disciplines
U1 = lambda x0,xi,u: np.array([np.log(x0[0]+1e-6) + np.log(xi[0]+1e-6) + np.log(u[1][0]+1e-6)])
U2 = lambda x0,xi,u: np.array([1/(x0[0]+1e-6) + 1/(xi[0]+1e-6) + 1/(u[0][0]+1e-6)])
f = lambda x0,x,u: x0[0] + x[0][0] + u[0][0] + u[1][0]
g = lambda x0,x,u: x[1][0] + u[1][0] - 10 # is not using the shared variable (but could in practice)
g21 = lambda x0,xi,ui: xi[0] + ui[0] - 10 # is not using the shared variable (but could in practice)

# system optimization problem
f_global = f
g_global = [0,] # no global constraints
# sub-system 1 optimization problem
f1 = 0 # no local objective
g1 = [0,] # no local constraints
# sub-system 2 optimization problem
f2 = 0 # no local objective
g2 = [g21,]

f_local = [f1,f2]
g_local = [g1,g2]
disciplines = [U1,U2]

u0 = 1.0; v0 = 1.0; w0 = 1.0; a0 = 1.0; b0 = 1.0
x0_init = [u0,] # initial guess for shared variable (u)
xi_init = [[v0,],[w0,]] # initial guess for local variables (v and w)
u_init = [[a0,],[b0,]] # initial guess for coupling variables (a and b)

lbs = [0.0,]; ubs = [10.,] # bounds of shared variable (u)
lbi = [[0.0,], [0.0,]]; ubi = [[10.,], [10.,]] # bounds of local variables (v and w)
lbu = [[0.0,], [0.0,]]; ubu = [[10.,], [10.,]] # bounds of coupling variables (a and b)

opt_options = {
    "seed": 0, 
    "budget": 1000, 
    "tol": 1e-12, 
    "display": False, 
    "opportunistic": False
}
my_atc = ATC(disciplines=disciplines,f_global=f_global, 
    g_global=g_global,f_local=f_local,g_local=g_local,x0_init=x0_init,
    xi_init=xi_init,u_init=u_init,lbs=lbs,ubs=ubs,lbi=lbi,ubi=ubi,
    lbu=lbu,ubu=ubu,opt_options=opt_options,w0=1.0,v0=0.0,
    beta=1.1,gamma=0.1)
x0,cx0,xi,u,cu,phi,f = my_atc.run_atc(k_max=200,tol=1e-6)

ATC terminated


Wow that took a while. This is because two nested optimization problems were solved as part of the main system-level optimization problem.

In [11]:
print("===========================")
f_tex = r"$f(\mathbf{x}^*) = %.4f$" %(f)
display(Latex(f_tex)) # minimum
x_tex = r"$\mathbf{x}^*$ = $[%.4f~~%.4f~~%.4f]^\mathrm{T}$" %(x0[0],xi[0][0],xi[1][0])
display(Latex(x_tex)) # optimizer
u_tex = r"$\hat{\mathbf{u}}(\mathbf{x}^*)$ = $[%.4f~~%.4f]^\mathrm{T}$" %(u[0][0],u[1][0])
display(Latex(u_tex)) # coupling variables
c_tex = r"$\mathbf{g}_c(\mathbf{x}^*)$ = $[%.4f~~%.4f~~%.4f]^\mathrm{T}$" %(cx0[0],cu[0],cu[1])
display(Latex(c_tex)) # inconsistencies
g2_tex = r"$\mathbf{g}_2(\mathbf{x}^*)$ = $[%.4f]^\mathrm{T}$" %(g21(x0,xi,u[1]))
display(Latex(g2_tex)) # local constraint
# print(f_tex) # minimum
# print(x_tex) # optimizer
# print(u_tex) # coupling variables
# print(c_tex) # inconsistencies
# print(g2_tex) # local constraint

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We did not do better than IDF or MDF but we managed to reduce the problem size compared to IDF.
We also needed to execute the discipline analyses $\mathcal{U}_1$ and $\mathcal{U_2}$ many more times. `ATC` keeps track of how many times each discipline was called. We can examine this result below.

In [12]:
my_atc.n_evaluations

[7308, 4369, 5865]

The numbers above are quite large. A work around for this is to use *non-hierarchical analytical target cascading* which does away with the system-level optimization problem. This means that we need to solve two optimization problems during each outer loop instead of three. To understand how the optimization problems are nested let us examine the XDSM for the above implementation of ATC on our numerical example.

<center>

| <p style="text-align:center;"><img src="./xdsm/atc.png" alt="atc" width="950p" align="center"/></p> |
|:--:|
| Fig.10 ATC architecture of the numerical example.|
</center>

or more compactly and generally using the XDSM and block diagrams below:

<center>

| <p style="text-align:center;"><img src="./xdsm/atc_compact.png" alt="atc_c" width="850p" align="center"/></p> |
|:--:|
| <p style="text-align:center;"><img src="./images/ATC_diagram.png" alt="ATC_diag" width="850p" align="center"/></p> |
| Fig.11 ATC architecture for a generic problem.|
</center>